In [ ]:
import fitz
from nltk.tokenize import sent_tokenize
import pandas as pd
import requests
import pdfkit
import time
from tqdm.notebook import tqdm
import sys
import os

sys.path.insert(1, "../../config")


['C:\\Users\\krish\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip', '../../config', '../../', '../', '../config', '../config', 'C:\\Users\\krish\\AppData\\Local\\Programs\\Python\\Python313\\DLLs', 'C:\\Users\\krish\\AppData\\Local\\Programs\\Python\\Python313\\Lib', 'C:\\Users\\krish\\AppData\\Local\\Programs\\Python\\Python313', 'c:\\Users\\krish\\Work\\projects\\esg-disclosure-classifier\\venv', '', 'c:\\Users\\krish\\Work\\projects\\esg-disclosure-classifier\\venv\\Lib\\site-packages', 'c:\\Users\\krish\\Work\\projects\\esg-disclosure-classifier\\venv\\Lib\\site-packages\\win32', 'c:\\Users\\krish\\Work\\projects\\esg-disclosure-classifier\\venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\krish\\Work\\projects\\esg-disclosure-classifier\\venv\\Lib\\site-packages\\Pythonwin']


In [9]:
from config.paths import METADATA_FILE, RAW_PDF_DIR, TEXT_DIR

## Preprocessing

- Extract text & segment into sentences

- Manually label ~500 sentences into ESG categories (small gold standard)

In [10]:
def extract_all_text(pdf_path: os.PathLike, skip_pages=0):
    """Extracts all text from a PDF document."""
    with fitz.open(pdf_path) as doc:
        full_text = ""
        for page_num in range(skip_pages, doc.page_count):
            page = doc.load_page(page_num)
            full_text += page.get_text("text") 
    return full_text

- Read data from `/data/raw` dir 
- extract text from pdfs
- using `metadata-sample.xlsx` store into company folder and apt year folder

In [11]:
# read metadata sheet

metadata = pd.read_excel(METADATA_FILE)

metadata.head(1)

,filename,title,year,type,lang,pages,size,hash,company_name,company_srid,company_isin,company_country,company_exchange,company_mic,company_ticker,company_sector,report_edition,report_assurance,report_standards
0,2021_Gym Group PLC_ESG,Gym Group PLC 2021 Sustainability Report,2021,Sustainability Report,EN,24,4.22,ddca6bd3b0f049006dead132c4c3e51c,Gym Group PLC,GB00BZBX0P70,GB00BZBX0P70,United Kingdom,London Stock Exchange,XLON,GYM,"Arts, Entertainment, and Recreation",NaN,NaN,GRI; SASB


In [ ]:
# make dirs for storing extracted text

metadata.loc[:, ['company_name', 'filename', 'year']].apply(lambda row: os.makedirs(TEXT_DIR / row.company_name.strip() / str(row.year), exist_ok=True), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
248    None
249    None
250    None
251    None
252    None
Length: 253, dtype: object

In [43]:
for row in metadata.loc[:, ['company_name', 'filename', 'year']].itertuples():

    full_text = extract_all_text(RAW_PDF_DIR / (row.filename + '.pdf'))

    with open(TEXT_DIR / row.company_name.strip() / str(row.year) / (row.filename + '.txt'), 'w', encoding='utf-8') as txt_file:

        txt_file.write(full_text)
    